In [1]:
import joblib
import keras
import numpy as np

Using Theano backend.


In [2]:
complete_features_matrix = joblib.load('files for cv/Pickles/Condensed_Basic+Temporal+Recency/train.pkl')
complete_features_matrix

matrix([[ 0.8909911 ,  0.75005068,  0.7497068 , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.8909911 ,  0.75005068,  0.7497068 , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.8909911 ,  0.75005068,  0.7497068 , ...,  0.        ,
          0.        ,  0.        ],
        ..., 
        [ 0.88798993,  0.88168444,  0.88168444, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.88798993,  0.88168444,  0.88168444, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.88798993,  0.88168444,  0.88168444, ...,  0.        ,
          0.        ,  0.        ]])

In [3]:
labels = joblib.load('algebra_2005_2008_pickles/labels.compressed')

In [4]:
train_index, test_index = joblib.load('files for cv/Pickles/cv_indices')


In [6]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import ProgbarLogger

model = Sequential([
    Dense(14, input_dim=28),
    Activation('sigmoid'),
    Dense(1),
    Activation('sigmoid'),
])

model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['accuracy'])

In [7]:
history = model.fit(complete_features_matrix, labels, 
                    nb_epoch=3, batch_size=128, 
                    verbose=1, validation_split=0.25)

Train on 6362940 samples, validate on 2120980 samples
Epoch 1/3
6362940/6362940 [==============================] - 48s - loss: 0.0960 - acc: 0.8744 - val_loss: 0.0969 - val_acc: 0.8720
Epoch 2/3
6362940/6362940 [==============================] - 51s - loss: 0.0938 - acc: 0.8764 - val_loss: 0.0966 - val_acc: 0.8721
Epoch 3/3
6362940/6362940 [==============================] - 52s - loss: 0.0935 - acc: 0.8768 - val_loss: 0.0965 - val_acc: 0.8722


In [7]:
test = joblib.load('files for cv/Pickles/Condensed_Basic+Temporal+Recency/test.pkl')

In [8]:
probas = model.predict_proba(verbose=1, x=test, batch_size=32)

508032/508912 [============================>.] - ETA: 0s

In [9]:
probas

array([[ 0.77084881],
       [ 0.80919737],
       [ 0.78761005],
       ..., 
       [ 0.66839451],
       [ 0.8676281 ],
       [ 0.90479183]], dtype=float32)

In [10]:
ones = []
zeros = []
for prob in probas:
    if prob > 0.5:
        ones.append(prob)
    else:
        zeros.append(prob)
        
print len(ones), len(zeros)

441466 67446


In [11]:
import csv
with open('algebra_2008_2009_submissiona.txt', 'wb') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter='\t')
    spamwriter.writerow(['Row', 'Correct First Attempt'])
    for index, one_prob in enumerate(probas):
        spamwriter.writerow([index+1, float(one_prob)])